<a href="https://colab.research.google.com/github/rsha0812/EVA4-S9-/blob/master/QuizDNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from datetime import date
from datetime import time
from datetime import datetime

In [0]:
def main():
  today = date.today()
  print("Current Date/Time: ", datetime.now())

if __name__ == "__main__":
  main()

Current Date/Time:  2020-03-17 20:35:46.061068


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
from dataset import *
from trainingfile import *

In [0]:
trainloader, testloader, classes = getData()

Files already downloaded and verified
Files already downloaded and verified


In [0]:
import torch.nn as nn
import torch.nn.functional as F
dropout_value = 0.1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) 

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Dropout(dropout_value)
        ) 

        # TRANSITION BLOCK 1
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=20, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 24; rf = 3
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12; rf = 4

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=32, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(32),
            nn.Dropout(dropout_value)
        ) 
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=56, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(56),
            nn.Dropout(dropout_value)
        ) 
        # TRANSITION BLOCK 2
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=56, out_channels=56, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 24; rf = 3
        self.pool2 = nn.MaxPool2d(2, 2) 

        # CONVOLUTION BLOCK 3
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=56, out_channels=96, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(96),
            nn.Dropout(dropout_value)
        )
        self.convblock9 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=96, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(96),
            nn.Dropout(dropout_value)
        ) 
        self.convblock10 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(128),
            nn.Dropout(dropout_value)
        )
        
        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=3)
        ) # output_size = 1; rf = 30

        self.fc1 = nn.Linear(256*1*1, 10)


        

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.pool1(x)
        x = self.convblock8(x)
        x = self.convblock9(x)
        x = self.convblock10(x)
        x = self.gap(x)        
        x = F.relu(self.fc1(x))
        x = x.view(-1, 10)
        return x

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

cuda


RuntimeError: ignored

In [0]:
Training(30, model, device, trainloader, testloader)
ClassTestAccuracy(testloader, device, model, classes)